In [25]:
import pandas as pd

# EDA

In [26]:
train = pd.read_csv("train.csv")
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [27]:
pd.get_dummies(train["LotShape"], dtype=int)

def make_dummies(df, col):
    data = df.copy()
    dummy_df = pd.get_dummies(df[col], dtype=int)
    dummy_df["Id"] = df["Id"]
    data = data.drop(col, axis=1)
    data = pd.merge(data,dummy_df, on="Id")
    return data

In [28]:
new_df = make_dummies(train, "LotShape")
new_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LandContour,Utilities,LotConfig,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,IR1,IR2,IR3,Reg
0,1,60,RL,65.0,8450,Pave,NaN,Lvl,AllPub,Inside,...,0,2,2008,WD,Normal,208500,0,0,0,1
1,2,20,RL,80.0,9600,Pave,NaN,Lvl,AllPub,FR2,...,0,5,2007,WD,Normal,181500,0,0,0,1
2,3,60,RL,68.0,11250,Pave,NaN,Lvl,AllPub,Inside,...,0,9,2008,WD,Normal,223500,1,0,0,0
3,4,70,RL,60.0,9550,Pave,NaN,Lvl,AllPub,Corner,...,0,2,2006,WD,Abnorml,140000,1,0,0,0
4,5,60,RL,84.0,14260,Pave,NaN,Lvl,AllPub,FR2,...,0,12,2008,WD,Normal,250000,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Lvl,AllPub,Inside,...,0,8,2007,WD,Normal,175000,0,0,0,1
1456,1457,20,RL,85.0,13175,Pave,NaN,Lvl,AllPub,Inside,...,0,2,2010,WD,Normal,210000,0,0,0,1
1457,1458,70,RL,66.0,9042,Pave,NaN,Lvl,AllPub,Inside,...,2500,5,2010,WD,Normal,266500,0,0,0,1
1458,1459,20,RL,68.0,9717,Pave,NaN,Lvl,AllPub,Inside,...,0,4,2010,WD,Normal,142125,0,0,0,1


In [29]:
# Select object and boolean dtypes
non_numerical_df = train.select_dtypes(include=['object', 'boolean'])

# Get the column names
non_numerical_columns = non_numerical_df.columns.tolist()

len(non_numerical_columns)

43

In [30]:
def make_dummies(df, cols):
    data = df.copy()
    dummy_dfs = []
    for col in cols:
        dummy_df = pd.get_dummies(df[col], dtype=int)
        dummy_dfs.append(dummy_df)
        data = data.drop(col, axis=1)
    for dummy_df in dummy_dfs:
        data = pd.merge(data, dummy_df, left_index=True, right_index=True)
    return data

In [31]:
columns_dummy = non_numerical_columns
new_df = make_dummies(train, columns_dummy)
new_df

MergeError: Passing 'suffixes' which cause duplicate columns {'Stone_x'} is not allowed.

#TODO
<ul>
<li> Kolommen die bestaan uit categoriën opslitsen in n kollomen met 1 cat per kolom</li>

</ul>

In [ ]:
TODO

NameError: name 'TODO' is not defined